# Comparison of Model Performance

All models are compared based on their forecast performannce on the period from 2024-01-23 to 2025-02-28. (including two ends).

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import t
from scipy.stats import norm
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

- Forecast Error Metrics (MAE, RMSE, MASE)

- Diebold-Mariano Test (for forecast superiority)

- Visual Comparison (actual vs. predicted volatility)

One of the considerations was choosing a recursive estimation scheme over a rolling scheme in forecast estimation. While the latter is better suited for purposes like forecast comparison using DM test (which assumes that the differential is covariance stationary), the recursive estimation scheme is still favoured for practical reasons. This is because by expanding the training set timeframe, it uses more and more data to make later forecasts - parameter uncertainty is reduced, and forecast error may become less variable. Also, parameter estimation uncertainty typically constitutes a tiny fraction of forecast error variance, so reducing it would not noticeably violate stationarity. 

In [16]:
# Diebold Mariano (1995) EPA Test
def diebold_mariano(actual, forecast1, forecast2, h=3):
    """
    DM test for h-step-ahead forecasts.
    H0: Forecasts have equal accuracy.
    HA: Forecast 1 is more accurate than Forecast 2.
    """
    # Forecast errors
    e1 = actual - forecast1
    e2 = actual - forecast2
    
    # Loss differential (squared errors)
    d = e1**2 - e2**2
    
    # DM statistic (with Newey-West adjustment for autocorrelation)
    n = len(d)
    d_mean = np.mean(d)
    d_var = np.var(d, ddof=1)  # HAC adjustment not shown here (see next section)
    dm_stat = d_mean / np.sqrt(d_var / n)
    
    # Critical value (standard normal for large n)
    p_value = 2 * norm.sf(np.abs(dm_stat))  # Two-tailed test
    
    return dm_stat, p_value

## 1D-Ahead Forecasts

In [ ]:
for h in [1,3,7,30]:
    # load data
    garch = pd.read_csv(f'../res/GARCH(1,0,1)_{h}D.csv')['Predicted']
    har = pd.read_csv(f'../res/HAR(1, 7, 30)_{h}D.csv')['Pred']
    garch_svm = pd.read_csv(f'../res/GARCH-SVM_{h}D.csv')['Predicted']
    actual = pd.read_csv(f'../res/lnRV_test_{h}D.csv')['lnRV']

    # Mean Absolute Error (MASE)
    mae_garch = mean_absolute_error(actual, garch)
    mae_har = mean_absolute_error(actual, har)
    mae_garch_svm = mean_absolute_error(actual, garch_svm)

    # Mean Absolute Error (MASE)
    mae_garch = mean_absolute_error(actual, garch)
    mae_har = mean_absolute_error(actual, har)
    mae_garch_svm = mean_absolute_error(actual, garch_svm)

    # Root Mean Squared Error (RMSE)
    rmse_garch = np.sqrt(mean_squared_error(actual, garch))
    rmse_har = np.sqrt(mean_squared_error(actual, har))
    rmse_garch_svm = np.sqrt(mean_squared_error(actual, garch_svm))

    # Mean Absolute Percentage Error (MAPE)
    mape_garch = mean_absolute_percentage_error(actual, garch) * 100
    mape_har = mean_absolute_percentage_error(actual, har) * 100
    mape_garch_svm = mean_absolute_percentage_error(actual, garch_svm) * 100

    # Mean Absolute Scaled Error (MASE) - Lagged realized vol as benchmark
    naive_forecast = np.roll(actual, h)
    naive_forecast[:h] = np.nan  # Set the first h values to NaN
    mase_garch = mean_absolute_error(actual[h:], garch[h:]) / mean_absolute_error(actual[h:], naive_forecast[h:])
    mase_har = mean_absolute_error(actual[h:], har[h:]) / mean_absolute_error(actual[h:], naive_forecast[h:])
    mase_garch_svm = mean_absolute_error(actual[h:], garch_svm[h:]) / mean_absolute_error(actual[h:], naive_forecast[h:])

    # R-squared
    r2_garch = r2_score(actual, garch)
    r2_har = r2_score(actual, har)
    r2_garch_svm = r2_score(actual, garch_svm)

    # Diebold-Mariano test
    dm_stat_garch_har, p_value_garch_har = diebold_mariano(actual, garch, har)
    dm_stat_garch_svm, p_value_garch_svm = diebold_mariano(actual, garch, garch_svm)
    dm_stat_har_svm, p_value_har_svm = diebold_mariano(actual, har, garch_svm)

    # save results to csv
    results = pd.DataFrame({
        'Metric': ['MAE', 'RMSE', 'MAPE', 'MASE', 'R-squared'],
        'GARCH': [mae_garch, rmse_garch, mape_garch, mase_garch, r2_garch],
        'HAR': [mae_har, rmse_har, mape_har, mase_har, r2_har],
        'GARCH-SVM': [mae_garch_svm, rmse_garch_svm, mape_garch_svm, mase_garch_svm, r2_garch_svm],
    })
    results.to_csv(f'../eval/metrics_{h}D.csv', index=False)

    # save DM test results to csv
    dm_results = pd.DataFrame({
        'Comparison': ['GARCH vs HAR', 'GARCH vs GARCH-SVM', 'HAR vs GARCH-SVM'],
        'DM Statistic': [dm_stat_garch_har, dm_stat_garch_svm, dm_stat_har_svm],
        'p-value': [p_value_garch_har, p_value_garch_svm, p_value_har_svm]
    })
    dm_results.to_csv(f'../eval/DM_test_{h}D.csv', index=False)